# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from absa_models.ABSA_BERT_Dropout_CNN_BiLSTM_Linear import ABSA_BERT_Dropout_CNN_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ABSA_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,absa_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
SEQ_LEN = 512

In [11]:
BERT_FINE_TUNED_PATH = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned.pth'

In [12]:
MODEL_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_cnn_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_cnn_bilstm_linear.csv'

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [14]:
best_accuracy = 0.0

In [15]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [16]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 4), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [17]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [18]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [19]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ABSA_BERT_Dropout_CNN_BiLSTM_Linear(torch.load(BERT_FINE_TUNED_PATH), bert_seq_len=SEQ_LEN, dropout=0.3, bilstm_in_features=256, conv_out_channels=SEQ_LEN, conv_kernel_size=3, no_out_labels=4, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.3791313171386719
Epoch: 0/2, Batch: 50/501, Loss: 0.7832396626472473
Epoch: 0/2, Batch: 100/501, Loss: 0.25685885548591614
Epoch: 0/2, Batch: 150/501, Loss: 0.16308583319187164
Epoch: 0/2, Batch: 200/501, Loss: 0.07236038148403168
Epoch: 0/2, Batch: 250/501, Loss: 0.053919460624456406
Epoch: 0/2, Batch: 300/501, Loss: 0.055026598274707794
Epoch: 0/2, Batch: 350/501, Loss: 0.03827543929219246
Epoch: 0/2, Batch: 400/501, Loss: 0.04102105274796486
Epoch: 0/2, Batch: 450/501, Loss: 0.06114530563354492
Epoch: 0/2, Batch: 500/501, Loss: 0.0535225048661232
Epoch: 1/2, Batch: 0/501, Loss: 0.03238680958747864
Epoch: 1/2, Batch: 50/501, Loss: 0.0463700145483017
Epoch: 1/2, Batch: 100/501, Loss: 0.02261694334447384
Epoch: 1/2, Batch: 150/501, Loss: 0.0643007904291153
Epoch: 1/2, Batch: 200/501, Loss: 0.035171955823898315
Epoch: 1/2, Batch: 250/501, Loss: 0.048358090221881866
Epoch: 1/2, Batch: 300/501, Loss: 0.04689274728298187
Epoch: 1/2, Batch: 350/501

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/501, Loss: 1.3641107082366943
Epoch: 0/2, Batch: 50/501, Loss: 0.8239360451698303
Epoch: 0/2, Batch: 100/501, Loss: 0.31828704476356506
Epoch: 0/2, Batch: 150/501, Loss: 0.16850361227989197
Epoch: 0/2, Batch: 200/501, Loss: 0.11828004568815231
Epoch: 0/2, Batch: 250/501, Loss: 0.09411289542913437
Epoch: 0/2, Batch: 300/501, Loss: 0.0803120881319046
Epoch: 0/2, Batch: 350/501, Loss: 0.05951399728655815
Epoch: 0/2, Batch: 400/501, Loss: 0.07759344577789307
Epoch: 0/2, Batch: 450/501, Loss: 0.06753435730934143
Epoch: 0/2, Batch: 500/501, Loss: 0.042851801961660385
Epoch: 1/2, Batch: 0/501, Loss: 0.05055856704711914
Epoch: 1/2, Batch: 50/501, Loss: 0.04003523290157318
Epoch: 1/2, Batch: 100/501, Loss: 0.05058004707098007
Epoch: 1/2, Batch: 150/501, Loss: 0.06702111661434174
Epoch: 1/2, Batch: 200/501, Loss: 0.046212080866098404
Epoch: 1/2, Batch: 250/501, Loss: 0.03653896227478981
Epoch: 1/2, Batch: 300/501, Loss: 0.045877035707235336
Epoch: 1/2, Batch: 350/50

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/501, Loss: 1.3971620798110962
Epoch: 0/2, Batch: 50/501, Loss: 0.6693621277809143
Epoch: 0/2, Batch: 100/501, Loss: 0.1958797723054886
Epoch: 0/2, Batch: 150/501, Loss: 0.11131821572780609
Epoch: 0/2, Batch: 200/501, Loss: 0.06754472851753235
Epoch: 0/2, Batch: 250/501, Loss: 0.05422203615307808
Epoch: 0/2, Batch: 300/501, Loss: 0.07840702682733536
Epoch: 0/2, Batch: 350/501, Loss: 0.034936219453811646
Epoch: 0/2, Batch: 400/501, Loss: 0.030279647558927536
Epoch: 0/2, Batch: 450/501, Loss: 0.017139554023742676
Epoch: 0/2, Batch: 500/501, Loss: 0.026397010311484337
Epoch: 1/2, Batch: 0/501, Loss: 0.03282485157251358
Epoch: 1/2, Batch: 50/501, Loss: 0.03176300972700119
Epoch: 1/2, Batch: 100/501, Loss: 0.030062565580010414
Epoch: 1/2, Batch: 150/501, Loss: 0.0178560558706522
Epoch: 1/2, Batch: 200/501, Loss: 0.015436606481671333
Epoch: 1/2, Batch: 250/501, Loss: 0.008604682981967926
Epoch: 1/2, Batch: 300/501, Loss: 0.027908919379115105
Epoch: 1/2, Batch: 35

...

Run 6/10
Epoch: 0/2, Batch: 0/501, Loss: 1.423202395439148
Epoch: 0/2, Batch: 50/501, Loss: 0.9547160863876343
Epoch: 0/2, Batch: 100/501, Loss: 0.369901567697525
Epoch: 0/2, Batch: 150/501, Loss: 0.1928642839193344
Epoch: 0/2, Batch: 200/501, Loss: 0.1265455037355423
Epoch: 0/2, Batch: 250/501, Loss: 0.10181789100170135
Epoch: 0/2, Batch: 300/501, Loss: 0.07552804797887802
Epoch: 0/2, Batch: 350/501, Loss: 0.06070491299033165
Epoch: 0/2, Batch: 400/501, Loss: 0.05331553891301155
Epoch: 0/2, Batch: 450/501, Loss: 0.05608479678630829
Epoch: 0/2, Batch: 500/501, Loss: 0.04431859403848648
Epoch: 1/2, Batch: 0/501, Loss: 0.04429730027914047
Epoch: 1/2, Batch: 50/501, Loss: 0.034737616777420044
Epoch: 1/2, Batch: 100/501, Loss: 0.034711193293333054
Epoch: 1/2, Batch: 150/501, Loss: 0.04101503640413284
Epoch: 1/2, Batch: 200/501, Loss: 0.042126771062612534
Epoch: 1/2, Batch: 250/501, Loss: 0.026310313493013382
Epoch: 1/2, Batch: 300/501, Loss: 0.028388500213623047
Epoch: 1/2, Batch: 350/501,

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 7/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4553301334381104
Epoch: 0/2, Batch: 50/501, Loss: 1.0866166353225708
Epoch: 0/2, Batch: 100/501, Loss: 0.4185488224029541
Epoch: 0/2, Batch: 150/501, Loss: 0.18107494711875916
Epoch: 0/2, Batch: 200/501, Loss: 0.12448842078447342
Epoch: 0/2, Batch: 250/501, Loss: 0.07927144318819046
Epoch: 0/2, Batch: 300/501, Loss: 0.06825843453407288
Epoch: 0/2, Batch: 350/501, Loss: 0.07052507996559143
Epoch: 0/2, Batch: 400/501, Loss: 0.05284758284687996
Epoch: 0/2, Batch: 450/501, Loss: 0.050642289221286774
Epoch: 0/2, Batch: 500/501, Loss: 0.056155432015657425
Epoch: 1/2, Batch: 0/501, Loss: 0.050901468843221664
Epoch: 1/2, Batch: 50/501, Loss: 0.04834197461605072
Epoch: 1/2, Batch: 100/501, Loss: 0.04309484362602234
Epoch: 1/2, Batch: 150/501, Loss: 0.03750821202993393
Epoch: 1/2, Batch: 200/501, Loss: 0.031653404235839844
Epoch: 1/2, Batch: 250/501, Loss: 0.04852695018053055
Epoch: 1/2, Batch: 300/501, Loss: 0.03347618132829666
Epoch: 1/2, Batch: 350/5

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 8/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4070982933044434
Epoch: 0/2, Batch: 50/501, Loss: 0.6454086899757385
Epoch: 0/2, Batch: 100/501, Loss: 0.21360935270786285
Epoch: 0/2, Batch: 150/501, Loss: 0.09324800223112106
Epoch: 0/2, Batch: 200/501, Loss: 0.06844113767147064
Epoch: 0/2, Batch: 250/501, Loss: 0.03397832438349724
Epoch: 0/2, Batch: 300/501, Loss: 0.032687027007341385
Epoch: 0/2, Batch: 350/501, Loss: 0.0403105653822422
Epoch: 0/2, Batch: 400/501, Loss: 0.02016087993979454
Epoch: 0/2, Batch: 450/501, Loss: 0.07819543033838272
Epoch: 0/2, Batch: 500/501, Loss: 0.022587209939956665
Epoch: 1/2, Batch: 0/501, Loss: 0.02245577983558178
Epoch: 1/2, Batch: 50/501, Loss: 0.023781055584549904
Epoch: 1/2, Batch: 100/501, Loss: 0.028993265703320503
Epoch: 1/2, Batch: 150/501, Loss: 0.030787330120801926
Epoch: 1/2, Batch: 200/501, Loss: 0.015860142186284065
Epoch: 1/2, Batch: 250/501, Loss: 0.031248364597558975
Epoch: 1/2, Batch: 300/501, Loss: 0.013999651186168194
Epoch: 1/2, Batch: 3

In [20]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.994969,0.994969,0.248751,0.994969,0.250000,0.994969,0.249374,499.831584
1,0.995129,0.995129,0.248782,0.995129,0.250000,0.995129,0.249390,494.516235
2,0.995098,0.995098,0.483781,0.995098,0.255878,0.995098,0.260355,486.927129
3,0.994664,0.994664,0.373733,0.994664,0.250226,0.994664,0.249850,468.425592
4,0.995074,0.995074,0.436274,0.995074,0.252411,0.995074,0.254151,403.872148
5,0.994848,0.994848,0.248712,0.994848,0.250000,0.994848,0.249354,356.734167
6,0.994957,0.994957,0.248739,0.994957,0.250000,0.994957,0.249368,356.549546
7,0.995184,0.995184,0.515532,0.995184,0.277121,0.995184,0.271461,356.531179
8,0.994891,0.994891,0.470177,0.994891,0.257213,0.994891,0.263361,444.991704
9,0.994871,0.994871,0.248727,0.994871,0.249999,0.994871,0.249362,491.557349


In [21]:
results.to_csv(STATS_OUTPUT)